# Automated Machine Learning の説明の確認

機械学習がますます普及するにつれて、モデルによる予測は私たちの社会の多くの側面に大きな影響を与えます。たとえば、機械学習モデルは、銀行が融資を許可するか、医師が治療の優先順位を決定する方法においてますます重要な要素となっています。モデルを解釈して説明する能力はますます重要になり、機械学習モデルによる予測の根拠を説明し、正当化し、モデル内の不注意な偏りを特定することができます。

自動機械学習を使用してモデルをトレーニングする場合、各機能がラベル予測に影響を与える程度を定量化する機能の重要性の説明を生成するオプションがあります。このラボでは、自動 Machine Learning 実験によって生成された説明について説明します。

## ワークスペースに接続する

まず、Azure ML SDK を使用してワークスペースに接続する必要があります。

> **注**: 前回の演習を完了してから Azure サブスクリプションとの認証済みセッションの有効期限が切れている場合は、再認証を求めるメッセージが表示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## コンピューティング先を準備する

以前のラボで作成した Azure Machine Learning コンピューティング クラスターを使用します (これがない場合は作成されます)。

> **重要**: 実行する前に、*コンピューティング クラスター*を以下のコードでコンピューティング クラスターの名前に変更してください。クラスター名は、長さが 2 〜 16 文字のグローバルに一意の名前である必要があります。英字、数字、- の文字が有効です。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

## 自動 Machine Learning 実験を実行する

このラボの時間を短縮するために、3 回の反復のみで自動 Machine Learning 実験を実行します。

**model_explainability** 構成オプションが **True** に設定されていることに注意してください。

In [ ]:
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails
from azureml.core import Dataset

try:
    # Prepare data for training
    default_ds = ws.get_default_datastore()
    if 'diabetes dataset' not in ws.datasets:
        default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                            target_path='diabetes-data/', # Put it in a folder path in the datastore
                            overwrite=True, # Replace existing files of the same name
                            show_progress=True)

        # Create a tabular dataset from the path on the datastore (this may take a short while)
        tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

        # Register the tabular dataset
        try:
            tab_data_set = tab_data_set.register(workspace=ws, 
                                    name='diabetes dataset',
                                    description='diabetes data',
                                    tags = {'format':'CSV'},
                                    create_new_version=True)
            print('Dataset registered.')
        except Exception as ex:
            print(ex)
    else:
        print('Dataset already registered.')
    train_data = ws.datasets.get("diabetes dataset")


    # Configure Auto ML
    automl_config = AutoMLConfig(name='Automated ML Experiment',
                                task='classification',
                                compute_target=training_cluster,
                                training_data = train_data,
                                n_cross_validations = 2,
                                label_column_name='Diabetic',
                                iterations=3,
                                primary_metric = 'AUC_weighted',
                                max_concurrent_iterations=3,
                                featurization='off',
                                model_explainability=True # Generate feature importance!
                                )

    # Run the Auto ML experiment
    print('Submitting Auto ML experiment...')
    automl_experiment = Experiment(ws, 'diabetes_automl')
    automl_run = automl_experiment.submit(automl_config)
    automl_run.wait_for_completion(show_output=True)
    RunDetails(automl_run).show()

except Exception as ex:
    print(ex)

## 特徴の重要度を表示する

上のウィジェットで実験が完了したら、最適な結果を生成した実行をクリックして詳細を表示します。次に、視覚化の下部までスクロールして、特徴の相対的な重要度を確認します。


## Azure Machine Learning Studio でモデルの説明を表示する

実験の実行が完了したら、ウィジェット内のリンクをクリックして Azure Machine Learning Studio で実行を確認し、**説明**タブを表示します。次に以下を実行します。

1. 自動 Machine Learning によって作成された Explainer を選択します。
2. 全体的なグローバル特徴の重要度を示す **グローバル重要度** グラフを表示します。
3. **サマリー重要度** グラフを表示すると、*スウォーム*、*バイオリン*、*ボックス* プロットのテスト データの各データ ポイントが表示されます。
4. 個々のポイントを選択すると、選択したデータ ポイントの個々の予測の **ローカル特徴の重要度** が表示されます。

> **詳細情報**: 自動 Machine Learning の詳細については、[Azure ML のドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-machine-learning-interpretability-automl)を参照してください。